In [1]:
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [35]:
from pywr.model import Model
from pywr.recorders import DeficitFrequencyNodeRecorder
from pywr.recorders import NumpyArrayParameterRecorder
my_model = Model.load('2_reservoirs.json')
my_model.run()

,Value
timestep,2199-12-27 00:00:00
timesteps,5218
time_taken,2.10514
time_taken_before,0.151433
time_taken_after,0.808167
time_taken_with_overhead,2.15502
speed,2478.22
num_scenarios,1
solver_name,glpk
version,1.10.0


In [1]:
def get_model_data(harmonic=True):

    with open('2_reservoir.json') as fh:
        data = json.load(fh)

    return data

In [2]:
def platypus_main(harmonic=False):
    import platypus
    from pywr.optimisation.platypus import PlatypusWrapper
    wrapper = PlatypusWrapper(get_model_data(harmonic=harmonic))
    with platypus.ProcessPoolEvaluator() as evaluator:
        algorithm = platypus.NSGAIII(wrapper.problem, population_size=50, evaluator=evaluator, divisions_outer=12)
        algorithm.run(10000)

    objectives = pd.DataFrame(data=np.array([s.objectives for s in algorithm.result]),
                              columns=[o.name for o in wrapper.model_objectives])

    title = 'harmonic' if harmonic else 'monthly'
    objectives.to_csv('C:/Users/robwd/two_reservoir_moea.csv', f'platypus_{title}')

    print(objectives)